In [1]:
import pandas as pd
import requests

USPTO_ALL_CBSA_URL = "https://www.uspto.gov/web/offices/ac/ido/oeip/taf/cls_cbsa/allcbsa_gd.htm"

def scrape_uspto_cbsa_patents(year: int, out_csv: str = None) -> pd.DataFrame:
    if year < 2000 or year > 2015:
        raise ValueError("This USPTO CBSA table supports years 2000–2015 only (not 2020).")

    html = requests.get(USPTO_ALL_CBSA_URL, timeout=30)
    html.raise_for_status()

    df = pd.read_html(html.text)[0].copy()

    # ✅ Normalize column names
    df.columns = [str(c).strip().upper() for c in df.columns]

    name_col = "U.S. REGIONAL TITLE"
    year_col = str(year)

    if name_col not in df.columns:
        raise RuntimeError(f"Name column not found. Columns: {df.columns.tolist()}")
    if year_col not in df.columns:
        raise RuntimeError(f"Year column {year_col} not found. Columns: {df.columns.tolist()}")

    out = df[[name_col, year_col]].rename(
        columns={name_col: "cbsa_name", year_col: "patent_count"}
    ).copy()

    out["cbsa_name"] = out["cbsa_name"].astype(str).str.strip()
    out["patent_count"] = pd.to_numeric(out["patent_count"], errors="coerce").fillna(0).astype(int)
    out["year"] = year

    # remove totals / blank rows
    out = out[(out["cbsa_name"] != "") & (out["cbsa_name"].str.upper() != "TOTAL")].reset_index(drop=True)

    if out_csv:
        out.to_csv(out_csv, index=False)

    return out

# Create the DataFrame in-memory; avoid writing intermediate CSV here
df_2015 = scrape_uspto_cbsa_patents(2015)
# print(df_2015.head())
# print("Rows:", len(df_2015))


/var/folders/l7/93rpm4x14fv6n998658ffrlc0000gn/T/ipykernel_10590/4113713486.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html.text)[0].copy()


In [2]:
import pandas as pd
import re

# 1) Load (prefer in-memory `df_2015` if available)
try:
    df = df_2015.copy()
except NameError:
    df = pd.read_csv("uspto_cbsa_patents_2015.csv")

# 2) Basic cleanup
df = df.copy()
df["cbsa_name"] = df["cbsa_name"].astype(str)

# strip whitespace + collapse multiple spaces
df["cbsa_name"] = (
    df["cbsa_name"]
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# normalize dash characters (sometimes tables contain weird unicode dashes)
df["cbsa_name"] = df["cbsa_name"].str.replace("–", "-", regex=False).str.replace("—", "-", regex=False)

# 3) Remove junk rows (blank names, "TOTAL", etc.)
df = df[df["cbsa_name"].ne("")]
df = df[~df["cbsa_name"].str.upper().eq("TOTAL")]

# 4) Ensure numeric patent_count
df["patent_count"] = pd.to_numeric(df["patent_count"], errors="coerce").fillna(0).astype(int)

# remove negatives if any (shouldn't happen, but safe)
df.loc[df["patent_count"] < 0, "patent_count"] = 0

# 5) Ensure year is int
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
df = df.dropna(subset=["year"])
df["year"] = df["year"].astype(int)

# 6) Create helper: extract state codes from the end (e.g., "... , CA" or "... , MA-NH")
# This is useful later if you need QA or disambiguation.
def extract_states(name: str):
    # capture everything after last comma as the "state part"
    parts = name.rsplit(",", 1)
    if len(parts) < 2:
        return None
    state_part = parts[1].strip()
    # keep only uppercase letters and hyphens
    state_part = re.sub(r"[^A-Z\-]", "", state_part.upper())
    return state_part if state_part else None

df["state_codes"] = df["cbsa_name"].apply(extract_states)

# 7) De-duplicate names (should not be needed, but safe)
# If duplicates exist, sum their patents.
df = (
    df.groupby(["cbsa_name", "year"], as_index=False)["patent_count"]
      .sum()
)

# 8) Quick sanity checks
print(df.head())
print("Rows:", len(df))
print("Patent count min/max:", df["patent_count"].min(), df["patent_count"].max())
print("Top 5 MSAs:\n", df.sort_values("patent_count", ascending=False).head(5))

# # 9) Save cleaned version
# df.to_csv("uspto_cbsa_patents_2015_clean.csv", index=False)
# print("Saved: uspto_cbsa_patents_2015_clean.csv")


        cbsa_name  year  patent_count
0  -- Subtotal --  2015        140929
1   Abbeville, LA  2015             1
2    Aberdeen, SD  2015             5
3    Aberdeen, WA  2015             4
4     Abilene, TX  2015             8
Rows: 1021
Patent count min/max: 0 140929
Top 5 MSAs:
                                              cbsa_name  year  patent_count
0                                       -- Subtotal --  2015        140929
897                                   TOTAL, ALL AREAS  2015        140928
811                 San Jose-Sunnyvale-Santa Clara, CA  2015         14618
809                  San Francisco-Oakland-Fremont, CA  2015          9732
615  New York-Northern New Jersey-Long Island, NY-N...  2015          7754


In [3]:
import pandas as pd
import re

# --- Load your cleaned patents file ---
pat = df.copy()

# --- Load census gazetteer (tab-delimited) ---
gaz = pd.read_csv("2023_Gaz_cbsa_national.txt", sep="\t", dtype={"GEOID": str})

# Keep only relevant columns, normalize
gaz = gaz.rename(columns={"GEOID": "cbsa", "NAME": "cbsa_name_ref"})
gaz["cbsa_name_ref"] = gaz["cbsa_name_ref"].astype(str).str.strip()

# ---------------------------
# Normalization helpers
# ---------------------------
def norm_name(s: str) -> str:
    s = str(s).upper().strip()
    s = s.replace("–", "-").replace("—", "-")
    s = re.sub(r"\s+", " ", s)

    # Remove common suffix differences:
    # Datawrapper/Census often has "Metro Area" / "Micro Area", USPTO sometimes doesn't
    s = s.replace(" METROPOLITAN STATISTICAL AREA", "")
    s = s.replace(" MICROPOLITAN STATISTICAL AREA", "")
    s = s.replace(" METRO AREA", "")
    s = s.replace(" MICRO AREA", "")

    # Remove periods and extra punctuation
    s = re.sub(r"[\.]", "", s)

    return s

pat["name_norm"] = pat["cbsa_name"].apply(norm_name)
gaz["name_norm"] = gaz["cbsa_name_ref"].apply(norm_name)

# ---------------------------
# Join patents to CBSA codes
# ---------------------------
merged = pat.merge(
    gaz[["cbsa", "cbsa_name_ref", "name_norm"]],
    on="name_norm",
    how="left"
)

# Report unmatched rows (important!)
unmatched = merged[merged["cbsa"].isna()][["cbsa_name", "patent_count"]]
print("Unmatched rows:", len(unmatched))
print(unmatched.head(20))

# Keep only matched for Datawrapper
dw = merged.dropna(subset=["cbsa"]).copy()

# Ensure CBSA is string (Datawrapper likes text IDs)
dw["cbsa"] = dw["cbsa"].astype(str).str.zfill(5)

# Final columns for Datawrapper
dw_final = dw[["cbsa", "cbsa_name", "patent_count", "year"]].rename(
    columns={"cbsa_name": "msa_name"}
)

# Optional: create a label column for top 5
dw_final["rank"] = dw_final["patent_count"].rank(method="dense", ascending=False).astype(int)
dw_final["top5_label"] = dw_final.apply(
    lambda r: r["msa_name"] if r["rank"] <= 5 else "",
    axis=1
)





Unmatched rows: 262
                               cbsa_name  patent_count
0                         -- Subtotal --        140929
1                          Abbeville, LA             1
7    Aguadilla-Isabela-San Sebasti�n, PR             3
11                    Albany-Lebanon, OR            41
21                           Allegan, MI            26
32                          Anderson, IN            33
33                          Anderson, SC            45
44                         Ashtabula, OH             6
51    Atlanta-Sandy Springs-Marietta, GA          2143
62                  Baltimore-Towson, MD           846
68                           Bastrop, LA             1
85                     Bennettsville, SC             0
87                         Berlin, NH-VT             1
92                 Birmingham-Hoover, AL           112
98                Bloomington-Normal, IL            74
103                 Boise City-Nampa, ID           722
105                            Boone, IA     

In [4]:
import pandas as pd

try:
    df = df_2015.copy()
except NameError:
    df = pd.read_csv("uspto_cbsa_patents_2015.csv")
print(df.columns)
print(df.head())


Index(['cbsa_name', 'patent_count', 'year'], dtype='object')
                                           cbsa_name  patent_count  year
0                 San Jose-Sunnyvale-Santa Clara, CA         14618  2015
1  New York-Northern New Jersey-Long Island, NY-N...          7754  2015
2                  San Francisco-Oakland-Fremont, CA          9732  2015
3               Los Angeles-Long Beach-Santa Ana, CA          6476  2015
4                     Boston-Cambridge-Quincy, MA-NH          5949  2015


In [5]:
import pandas as pd
import re

# your data (prefer in-memory)
try:
    pat = df_2015.copy()
except NameError:
    pat = pd.read_csv("uspto_cbsa_patents_2015.csv")

# gazetteer (update path/filename as needed)
gaz = pd.read_csv("2023_Gaz_cbsa_national.txt", sep="\t", dtype={"GEOID": str})

gaz = gaz.rename(columns={"GEOID": "cbsa", "NAME": "cbsa_name_ref"})
gaz["cbsa_name_ref"] = gaz["cbsa_name_ref"].astype(str).str.strip()

def norm(s):
    s = str(s).upper().strip()
    s = s.replace("–", "-").replace("—", "-")
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"\.", "", s)
    return s

pat["key"] = pat["cbsa_name"].apply(norm)
gaz["key"] = gaz["cbsa_name_ref"].apply(norm)

merged = pat.merge(gaz[["cbsa", "cbsa_name_ref", "key"]], on="key", how="left")

# Keep only matched rows
dw = merged.dropna(subset=["cbsa"]).copy()
dw["cbsa"] = dw["cbsa"].astype(str).str.zfill(5)

# Top-5 label column (for Datawrapper labels/tooltips)
dw["rank"] = dw["patent_count"].rank(method="dense", ascending=False).astype(int)
dw["top5_label"] = dw.apply(lambda r: r["cbsa_name"] if r["rank"] <= 5 else "", axis=1)

dw_final = dw[["cbsa", "cbsa_name", "patent_count", "year", "top5_label"]].rename(
    columns={"cbsa_name": "msa_name"}
)

# keep `dw_final` in-memory for downstream steps; do not write intermediate CSV
# dw_final.to_csv("datawrapper_msa_patents_2015.csv", index=False)


In [6]:
import pandas as pd
import re
from difflib import get_close_matches

try:
    pat = df_2015.copy()
except NameError:
    pat = pd.read_csv("uspto_cbsa_patents_2015.csv")
gaz = pd.read_csv("2023_Gaz_cbsa_national.txt", sep="\t", dtype={"GEOID": str})
gaz = gaz.rename(columns={"GEOID": "cbsa", "NAME": "cbsa_name_ref"})

def norm(s):
    s = str(s).upper().strip()
    s = s.replace("–", "-").replace("—", "-")
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"\.", "", s)
    # remove Census suffixes
    s = s.replace(" METRO AREA", "")
    s = s.replace(" MICRO AREA", "")
    s = s.replace(" METROPOLITAN STATISTICAL AREA", "")
    s = s.replace(" MICROPOLITAN STATISTICAL AREA", "")
    return s.strip()

pat["key"] = pat["cbsa_name"].apply(norm)
gaz["key"] = gaz["cbsa_name_ref"].apply(norm)

# exact match merge first
merged = pat.merge(gaz[["cbsa", "cbsa_name_ref", "key"]], on="key", how="left")

# fuzzy match for remaining (best-effort)
gaz_keys = gaz["key"].tolist()
gaz_lookup = dict(zip(gaz["key"], gaz["cbsa"]))

def fuzzy_cbsa(pat_key):
    matches = get_close_matches(pat_key, gaz_keys, n=1, cutoff=0.92)
    return gaz_lookup[matches[0]] if matches else None

mask = merged["cbsa"].isna()
merged.loc[mask, "cbsa"] = merged.loc[mask, "key"].apply(fuzzy_cbsa)

print("Matches after fuzzy:", merged["cbsa"].notna().sum(), "out of", len(merged))

# Keep matched, format CBSA, and add top5 label
dw = merged.dropna(subset=["cbsa"]).copy()
dw["cbsa"] = dw["cbsa"].astype(str).str.zfill(5)

dw["rank"] = dw["patent_count"].rank(method="dense", ascending=False).astype(int)
dw["top5_label"] = dw.apply(lambda r: r["cbsa_name"] if r["rank"] <= 5 else "", axis=1)

dw_final = dw[["cbsa", "cbsa_name", "patent_count", "year", "top5_label"]].rename(
    columns={"cbsa_name": "msa_name"}
)




Matches after fuzzy: 762 out of 1027


In [7]:
import pandas as pd

try:
    df = dw_final.copy()
except NameError:
    try:
        df = df_final.copy()
    except NameError:
        df = pd.read_csv("datawrapper_msa_patents_2015.csv", dtype={"cbsa": str})

# keep only what Datawrapper needs
df = df[["cbsa", "msa_name", "patent_count", "top5_label"]].copy()

# enforce clean formatting
df["cbsa"] = df["cbsa"].str.strip().str.zfill(5)
df["msa_name"] = df["msa_name"].astype(str).str.strip()
df["top5_label"] = df["top5_label"].astype(str).fillna("").str.strip()

# ensure patent_count is integer
df["patent_count"] = pd.to_numeric(df["patent_count"], errors="coerce").fillna(0).astype(int)

# optional: remove any duplicates by cbsa (keep max, or sum)
df = df.groupby(["cbsa", "msa_name"], as_index=False)["patent_count"].sum()

# recompute top5_label after grouping (important!)
df = df.sort_values("patent_count", ascending=False).reset_index(drop=True)
df["top5_label"] = ""
df.loc[:4, "top5_label"] = df.loc[:4, "msa_name"]

print(df.head(10))


    cbsa                                      msa_name  patent_count  \
0  41940            San Jose-Sunnyvale-Santa Clara, CA         14618   
1  41860             San Francisco-Oakland-Fremont, CA          9732   
2  42660                   Seattle-Tacoma-Bellevue, WA          4739   
3  33460       Minneapolis-St. Paul-Bloomington, MN-WI          3419   
4  19100               Dallas-Fort Worth-Arlington, TX          3026   
5  12420              Austin-Round Rock-San Marcos, TX          2700   
6  37980   Philadelphia-Camden-Wilmington, PA-NJ-DE-MD          2357   
7  47900  Washington-Arlington-Alexandria, DC-VA-MD-WV          2310   
8  38900           Portland-Vancouver-Hillsboro, OR-WA          2163   
9  39580                              Raleigh-Cary, NC          1535   

                                top5_label  
0       San Jose-Sunnyvale-Santa Clara, CA  
1        San Francisco-Oakland-Fremont, CA  
2              Seattle-Tacoma-Bellevue, WA  
3  Minneapolis-St. Paul-Blo

In [8]:
df["cbsa"] = df["cbsa"].astype(str).str.replace(",", "").str.zfill(5)


In [9]:

# keep minimal result in-memory for subsequent steps
df_minimal = df.copy()

In [10]:
import pandas as pd

try:
    df = df_minimal.copy()
except NameError:
    df = pd.read_csv("datawrapper_msa_patents_MINIMAL.csv", dtype={"cbsa": str})

# remove these 10 unmatched codes (replace list with the exact red codes you see)
bad = ["24180","30580","35340","42580","32060","36180","13860","18340","33380","46580"]

df = df[~df["cbsa"].isin(bad)]


In [11]:
import pandas as pd
import re

try:
    df = df_minimal.copy()
except NameError:
    df = pd.read_csv("datawrapper_msa_patents_MINIMAL.csv", dtype={"cbsa": str})

def clean_msa_name(s):
    s = str(s).strip()
    s = s.replace("–", "-").replace("—", "-")
    s = re.sub(r"\s+", " ", s)
    # ensure there's a space after commas
    s = re.sub(r",\s*", ", ", s)
    # remove any accidental code stuck at end (5 digits)
    s = re.sub(r"\s*\d{5}\s*$", "", s)
    return s

df["msa_name"] = df["msa_name"].apply(clean_msa_name)
df["top5_label"] = df["top5_label"].apply(clean_msa_name)

# keep cleaned-names version in-memory
df_cleaned_names = df.copy()




In [12]:
import pandas as pd

try:
    df = df_cleaned_names.copy()
except NameError:
    df = pd.read_csv("datawrapper_msa_patents_MINIMAL_clean_names.csv", dtype={"cbsa": str})

short_map = {
    "San Jose-Sunnyvale-Santa Clara, CA": "San Jose, CA",
    "San Francisco-Oakland-Fremont, CA": "San Francisco, CA",
    "Seattle-Tacoma-Bellevue, WA": "Seattle, WA",
    "Minneapolis-St. Paul-Bloomington, MN-WI": "Minneapolis–St. Paul, MN–WI",
    "Dallas-Fort Worth-Arlington, TX": "Dallas–Fort Worth, TX",
}

df["top5_short_label"] = df["top5_label"].map(short_map).fillna("")



In [13]:
import pandas as pd

# prefer in-memory cleaned names
try:
    df = df_cleaned_names.copy()
except NameError:
    df = pd.read_csv("datawrapper_msa_patents_MINIMAL_clean_names.csv", dtype={"cbsa": str})

# Replace actual NaN + literal "nan" with empty strings
df["top5_label"] = df["top5_label"].fillna("").astype(str)
df.loc[df["top5_label"].str.lower().eq("nan"), "top5_label"] = ""

# (Optional) also clean msa_name if needed
df["msa_name"] = df["msa_name"].fillna("").astype(str).str.strip()




In [14]:
import pandas as pd

# Prefer in-memory cleaned names or minimal frames before reading a file
try:
    df = df_cleaned_names.copy()
except NameError:
    try:
        df = df_minimal.copy()
    except NameError:
        df = pd.read_csv("datawrapper_msa_patents_TOP5labels_NO_NAN.csv", dtype={"cbsa": str})

short_map = {
    "San Jose-Sunnyvale-Santa Clara, CA": "San Jose, CA",
    "San Francisco-Oakland-Fremont, CA": "San Francisco, CA",
    "Seattle-Tacoma-Bellevue, WA": "Seattle, WA",
    "Minneapolis-St. Paul-Bloomington, MN-WI": "Minneapolis–St. Paul, MN–WI",
    "Dallas-Fort Worth-Arlington, TX": "Dallas–Fort Worth, TX",
}

df["top5_label"] = df["top5_label"].map(short_map).fillna("")

# expose this intermediate for later fallbacks
df_top5_no_nan = df.copy()


In [15]:

import pandas as pd

# Choose the most-recent in-memory DataFrame available, fall back to files
try:
    df = df_top5_short.copy()
except NameError:
    try:
        df = df_top5_no_nan.copy()
    except NameError:
        try:
            df = df_cleaned_names.copy()
        except NameError:
            try:
                df = df_minimal.copy()
            except NameError:
                try:
                    df = dw_final.copy()
                except NameError:
                    df = pd.read_csv("datawrapper_msa_patents_TOP5labels_SHORT.csv", dtype={"cbsa": str})

bad = {"24180","30500","35340","42580","32060","36180","13860","18340","33380","46580"}
df = df[~df["cbsa"].isin(bad)].copy()

# FINAL OUTPUT: write only here
df.to_csv("datawrapper_cleaned.csv", index=False)
print("Saved cleaned file with mismatched CBSAs removed. Rows:", len(df))


Saved cleaned file with mismatched CBSAs removed. Rows: 752
